# Extraction of the "Signaling by EGFR" pathway


#### Import libraries

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV, N3, XML, TURTLE
import IPython
import subprocess
import time
import os
import glob
from requests.utils import requote_uri
from urllib.parse import quote
import re
import rdflib
import pandas as pd

#### Define URL of SPARQL endpoint

In [2]:
endpointURL_reactome_standalone_biopax = "http://localhost:3030/reactome_standalone_biopax/query"
rdfFormat = "turtle"
current_directory = os.getcwd()
BioPAX_Ontology_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/BioPAX_Ontology', 'biopax-level3.owl')

#### Define RDF prefixes

In [3]:
reactomeVersion = 77 # actually it is version 90 (version in October, 2024), but 77 is mentioned in the URI 
prefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>

PREFIX chebi: <http://purl.obolibrary.org/obo/chebi/>
PREFIX chebidb: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX chebirel: <http://purl.obolibrary.org/obo/CHEBI#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

PREFIX bp3: <http://www.biopax.org/release/biopax-level3.owl#>

# Homo_sapiens-20170221.owl
#PREFIX reactome: <http://www.reactome.org/biopax/59/48887#> 
#
# Homo_sapiens-20210608.owl
#PREFIX reactome: <http://www.reactome.org/biopax/77/48887#>
#
# Homo_sapiens-20220614.owl
#PREFIX reactome: <http://www.reactome.org/biopax/81/48887#>
#
# Homo_sapiens-20221130.owl
#PREFIX reactome: <http://www.reactome.org/biopax/83/48887#>

PREFIX reactome: <http://www.reactome.org/biopax/{}/48887#>
""".format(reactomeVersion)

biopaxURI = "http://www.biopax.org/release/biopax-level3.owl#"

# Useful functions

In [4]:
def displaySparqlResults(results):
    '''
    Displays as HTML the result of a SPARQLWrapper query in a Jupyter notebook.
    
        Parameters:
            results (dictionnary): the result of a call to SPARQLWrapper.query().convert()
    '''
    variableNames = results['head']['vars']
    tableCode = '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format('</th><th>'.join(variableNames), '</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join([row[vName]['value'] if vName in row.keys() else "&nbsp;" for vName in variableNames]))for row in results["results"]["bindings"]))
    IPython.display.display(IPython.display.HTML(tableCode))

In [5]:
def getPathwayURIbyName(pathwayName):
    query="""
SELECT DISTINCT ?pathwayURI ?pathwayReactomeID
WHERE {
  ?pathwayURI bp3:name|bp3:displayName \"""" + pathwayName + """\" .
  ?pathwayURI bp3:xref [ rdf:type bp3:UnificationXref ;
                      bp3:db "Reactome" ;
                      bp3:id ?pathwayReactomeID ] .
}
"""
    sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
    sparql.setQuery(prefixes+query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results

In [ ]:
def extract_prefix_mappings(prefixes_string):
    """
    Extract prefix mappings from the SPARQL prefixes string.
    
    Parameters:
    prefixes_string (str): String containing PREFIX declarations
    
    Returns:
    dict: Mapping of full URIs to their prefixes
    """
    # Extract prefix declarations using regex
    prefix_pattern = re.compile(r'PREFIX\s+(\w+):\s*<([^>]+)>', re.IGNORECASE)
    return {uri: prefix for prefix, uri in prefix_pattern.findall(prefixes_string)}

def convert_to_prefixed_uri(uri_string, prefix_mappings):
    """
    Convert a full URI to prefixed format.
    
    Parameters:
    uri_string (str): Full URI string
    prefix_mappings (dict): Mapping of URIs to prefixes
    
    Returns:
    str: URI in prefixed format (e.g., 'reactome:Protein')
    """
    for uri_base, prefix in prefix_mappings.items():
        if uri_string.startswith(uri_base):
            local_part = uri_string[len(uri_base):]
            return f"{prefix}:{local_part}"
    return uri_string  # Return original if no prefix matches

def save_for_cytoscape(sparql, prefixes_string, output_file, format='csv', separator=','):
    """
    Save SPARQL CONSTRUCT results in a format compatible with Cytoscape,
    using prefix notation for URIs.
    
    Parameters:
    sparql (SPARQLWrapper): Configured SPARQLWrapper instance with query
    prefixes_string (str): String containing PREFIX declarations
    output_file (str): Path to save the output file
    format (str): Output format ('csv' or 'tsv')
    separator (str): Column separator (',' for CSV, '\t' for TSV)
    """
    # Extract prefix mappings
    prefix_mappings = extract_prefix_mappings(prefixes_string)
    
    # Get the results as an RDF graph
    sparql.setReturnFormat(TURTLE)
    results = sparql.queryAndConvert()
    
    # Create an RDFlib graph
    g = rdflib.Graph()
    if isinstance(results, bytes):
        g.parse(data=results.decode('utf-8'), format='turtle')
    else:
        g.parse(data=results, format='turtle')
    
    # Convert triples to a list of dictionaries with prefixed URIs
    triples_data = []
    for s, p, o in g:
        # Convert each URI to prefixed format
        subject = convert_to_prefixed_uri(str(s), prefix_mappings)
        predicate = convert_to_prefixed_uri(str(p), prefix_mappings)
        object_ = convert_to_prefixed_uri(str(o), prefix_mappings)
        
        triples_data.append({
            'Source': subject,
            'Interaction': predicate,
            'Target': object_
        })
    
    # Convert to DataFrame for easy CSV/TSV export
    df = pd.DataFrame(triples_data)
    
    # Save to file
    if format == 'csv':
        df.to_csv(output_file, index=False, sep=',')
    else:  # tsv
        df.to_csv(output_file, index=False, sep='\t')
    
    print(f"Saved {len(triples_data)} interactions to {output_file}")
    return df

def preview_network_data(df, n=5):
    """
    Preview the network data before importing into Cytoscape.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing the network data
    n (int): Number of rows to preview
    """
    print(f"\nPreview of network data ({len(df)} total interactions):")
    print(f"\nFirst {n} interactions:")
    print(df.head(n))
    
    # Print some basic network statistics
    unique_nodes = set(df['Source'].unique()) | set(df['Target'].unique())
    print(f"\nNetwork statistics:")
    print(f"Number of unique nodes: {len(unique_nodes)}")
    print(f"Number of interactions: {len(df)}")
    print(f"Unique interaction types:")
    for interaction in sorted(df['Interaction'].unique()):
        print(f"  - {interaction}")

# Launch SPARQL endpoint loading the standalone BioPAX export of Reactome

In [ ]:
ReactomeBioPAX_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/ReactomeBioPAX/', 'Homo_sapiens.owl')
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', ReactomeBioPAX_file_path,
    '--file', BioPAX_Ontology_file_path,
    '/reactome_standalone_biopax']


process = subprocess.Popen(command)
time.sleep(60)

# Extraction of "Signaling by EGFR" (R-HSA-177929) pathway from Reactome BioPAX export version 90

## 1 - Extract direct pathway components

In [ ]:
query_extract_pathway_compo = """ 
CONSTRUCT {
  ?pathwayStart bp3:pathwayComponent ?direct_component
}
WHERE {
  VALUES ?pathwayID { "R-HSA-177929" }
  ?pathwayIDxref bp3:id ?pathwayID .
  ?pathwayIDxref rdf:type bp3:UnificationXref .
  ?pathwayStart bp3:xref ?pathwayIDxref .
  ?pathwayStart rdf:type bp3:Pathway .
  
  ?pathwayStart bp3:pathwayComponent ?direct_component .
}
"""

sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
sparql.setQuery(prefixes+query_extract_pathway_compo)
df = save_for_cytoscape(sparql, prefixes, "../Results/1_direct_pathway_compo.csv", format='csv')
preview_network_data(df)

## 2 - Description of direct pathway components of type BiochemicalReaction

In [ ]:
query_direct_br = """ 
CONSTRUCT {
  ?direct_component ?relations_pathway_compo_br ?linked_pathway_compo_br
}
WHERE {
  VALUES ?pathwayID { "R-HSA-177929" }
  ?pathwayIDxref bp3:id ?pathwayID .
  ?pathwayIDxref rdf:type bp3:UnificationXref .
  ?pathwayStart bp3:xref ?pathwayIDxref .
  ?pathwayStart rdf:type bp3:Pathway .
  
  # Pathway components
  ?pathwayStart bp3:pathwayComponent ?direct_component .
  ?direct_component rdf:type bp3:BiochemicalReaction .
  VALUES ?relations_pathway_compo_br { bp3:displayName bp3:left bp3:right bp3:conversionDirection bp3:participantStoichiometry}
  ?direct_component ?relations_pathway_compo_br ?linked_pathway_compo_br .
}
""" 

sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
sparql.setQuery(prefixes+query_direct_br)
df = save_for_cytoscape(sparql, prefixes, "../Results/2_direct_pathway_compo_br.csv", format='csv')
preview_network_data(df)

## 3 - Description of direct pathway components of type Pathway

In [ ]:
query_direct_pathway = """ 
CONSTRUCT {
  ?direct_component ?relations_pathway_compo_path ?linked_pathway_compo_path
}
WHERE {
  VALUES ?pathwayID { "R-HSA-177929" }
  ?pathwayIDxref bp3:id ?pathwayID .
  ?pathwayIDxref rdf:type bp3:UnificationXref .
  ?pathwayStart bp3:xref ?pathwayIDxref .
  ?pathwayStart rdf:type bp3:Pathway .
  
  # Pathway components
  ?pathwayStart bp3:pathwayComponent ?direct_component .
  ?pathwayStart ?pathway_component ?direct_component .
  ?direct_component rdf:type bp3:Pathway .
  VALUES ?relations_pathway_compo_path { bp3:displayName bp3:pathwayComponent bp3:pathwayOrder }
  ?direct_component ?relations_pathway_compo_path ?linked_pathway_compo_path .
}
"""

sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
sparql.setQuery(prefixes+query_direct_pathway)
df = save_for_cytoscape(sparql, prefixes, "../Results/3_direct_pathway_compo_path.csv", format='csv')
preview_network_data(df)

## 4 - Extraction of direct pathway steps

In [ ]:
query_direct_ps = """ 
CONSTRUCT {
  ?pathwayStart bp3:pathwayOrder ?direct_ps
}
WHERE {
  VALUES ?pathwayID { "R-HSA-177929" }
  ?pathwayIDxref bp3:id ?pathwayID .
  ?pathwayIDxref rdf:type bp3:UnificationXref .
  ?pathwayStart bp3:xref ?pathwayIDxref .
  ?pathwayStart rdf:type bp3:Pathway .
  
  # DIRECT PATHWAY STEPS
  ?pathwayStart bp3:pathwayOrder ?direct_ps .
  ?direct_ps rdf:type bp3:PathwayStep .
}
"""

sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
sparql.setQuery(prefixes+query_direct_ps)
df = save_for_cytoscape(sparql, prefixes, "../Results/4_direct_pathway_steps.csv", format='csv')
preview_network_data(df)

## 5 - Description of direct pathway steps 

In [ ]:
query_description_direct_ps = """ 
CONSTRUCT {
  ?direct_ps ?relations_of_interest ?linked_to_ps
}
WHERE {
  VALUES ?pathwayID { "R-HSA-177929" }
  ?pathwayIDxref bp3:id ?pathwayID .
  ?pathwayIDxref rdf:type bp3:UnificationXref .
  ?pathwayStart bp3:xref ?pathwayIDxref .
  ?pathwayStart rdf:type bp3:Pathway .
  
  # DIRECT PATHWAY STEPS
  ?pathwayStart bp3:pathwayOrder ?direct_ps .
  ?direct_ps rdf:type bp3:PathwayStep .
  
  # RELATIONS OF PATHWAY STEPS
  VALUES ?relations_of_interest { bp3:stepProcess bp3:nextStep }
  ?direct_ps ?relations_of_interest ?linked_to_ps .
}
"""

sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
sparql.setQuery(prefixes+query_description_direct_ps)
df = save_for_cytoscape(sparql, prefixes, "../Results/5_direct_pathway_steps_next_steps_and_step_process.csv", format='csv')
preview_network_data(df)

## 6 - Description of step processes linked to pathway steps

In [ ]:
query_step_processes = """ 
CONSTRUCT {
  ?linked_to_ps ?relations_of_interest_starting_from_step_process ?step_process_relations
}
WHERE {
  VALUES ?pathwayID { "R-HSA-177929" }
  ?pathwayIDxref bp3:id ?pathwayID .
  ?pathwayIDxref rdf:type bp3:UnificationXref .
  ?pathwayStart bp3:xref ?pathwayIDxref .
  ?pathwayStart rdf:type bp3:Pathway .
  
  # DIRECT PATHWAY STEPS
  ?pathwayStart bp3:pathwayOrder ?direct_ps .
  ?direct_ps rdf:type bp3:PathwayStep .
  
  # RELATIONS OF PATHWAY STEPS
  VALUES ?relations_of_interest { bp3:stepProcess bp3:nextStep }
  ?direct_ps ?relations_of_interest ?linked_to_ps .
  VALUES ?relations_of_interest_starting_from_step_process { bp3:left bp3:pathwayOrder bp3:pathwayComponent bp3:right bp3:controller bp3:controlled bp3:displayName bp3:conversionDirection bp3:participantStoichiometry }
  ?linked_to_ps ?relations_of_interest_starting_from_step_process ?step_process_relations .
}
"""

sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
sparql.setQuery(prefixes+query_step_processes)
df = save_for_cytoscape(sparql, prefixes, "../Results/6_pathway_steps_steps_processes.csv", format='csv')
preview_network_data(df)

## 7 - Description of biochemical reactions linked to direct subpathways

In [ ]:
query_br_linked_to_ps = """ 
CONSTRUCT {
  ?linked_to_direct_compo ?relations_br ?description_br
}
WHERE {
  VALUES ?pathwayID { "R-HSA-177929" }
  ?pathwayIDxref bp3:id ?pathwayID .
  ?pathwayIDxref rdf:type bp3:UnificationXref .
  ?pathwayStart bp3:xref ?pathwayIDxref .
  ?pathwayStart rdf:type bp3:Pathway .
  VALUES ?pathway_compo_types { bp3:Pathway bp3:BiochemicalReaction }
  
  # DIRECT COMPONENTS
  ?pathwayStart bp3:pathwayComponent ?direct_component .
  ?pathwayStart ?pathway_component ?direct_component .
  ?direct_component rdf:type ?pathway_compo_types .
  
  # RELATIONS OF PATHWAY COMPONENTS
  VALUES ?relations_of_interest { bp3:left bp3:right bp3:displayName bp3:conversionDirection bp3:pathwayOrder bp3:pathwayComponent bp3:participantStoichiometry }
  ?direct_component ?relations_of_interest ?linked_to_direct_compo .
  
  ?linked_to_direct_compo rdf:type bp3:BiochemicalReaction .
  VALUES ?relations_br {bp3:left bp3:right bp3:displayName bp3:conversionDirection bp3:bp3:participantStoichiometry }
  ?linked_to_direct_compo ?relations_br ?description_br .
}
"""

sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
sparql.setQuery(prefixes+query_br_linked_to_ps)
df = save_for_cytoscape(sparql, prefixes, "../Results/7_linked_to_br_of_direct_pathway.csv", format='csv')
preview_network_data(df)

## 8 - Description of step processes of pathway steps linked to subpathways

In [ ]:
query_step_processes_subpathways = """ 
CONSTRUCT {
  ?pathwayStep bp3:stepProcess ?stepProcess
}
WHERE {
  VALUES ?pathwayID { "R-HSA-177929" }
  ?pathwayIDxref bp3:id ?pathwayID .
  ?pathwayIDxref rdf:type bp3:UnificationXref .
  ?pathwayStart bp3:xref ?pathwayIDxref .
  ?pathwayStart rdf:type bp3:Pathway .

  ?pathwayStart bp3:pathwayComponent ?pathwayCompo .
  ?pathwayCompo bp3:pathwayOrder ?pathwayStep .
  ?pathwayStep bp3:stepProcess ?stepProcess .
}
"""

sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
sparql.setQuery(prefixes+query_step_processes_subpathways)
df = save_for_cytoscape(sparql, prefixes, "../Results/8_processes_of_pathway_steps_linked_to_subpathways.csv", format='csv')
preview_network_data(df)

## 9 - Describe the processes linked to pathway steps linked to direct subpathways

In [ ]:
query_step_processes_subpathways_description = """  
CONSTRUCT {
  ?stepProcess ?properties_of_interest ?description_step_process
}
WHERE {
  VALUES ?pathwayID { "R-HSA-177929" }
  ?pathwayIDxref bp3:id ?pathwayID .
  ?pathwayIDxref rdf:type bp3:UnificationXref .
  ?pathwayStart bp3:xref ?pathwayIDxref .
  ?pathwayStart rdf:type bp3:Pathway .
  VALUES ?properties_of_interest { bp3:right bp3:left bp3:participantStoichiometry bp3:controlled bp3:controller bp3:displayName}

  ?pathwayStart bp3:pathwayComponent ?pathwayCompo .
  ?pathwayCompo bp3:pathwayOrder ?pathwayStep .
  ?pathwayStep bp3:stepProcess ?stepProcess .
  ?stepProcess ?properties_of_interest ?description_step_process .
}
"""

sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
sparql.setQuery(prefixes+query_step_processes_subpathways_description)
df = save_for_cytoscape(sparql, prefixes, "../Results/9_description_processes_pf_ps_linked_to_subpathways.csv", format='csv')
preview_network_data(df)

## 10 - Get next steps between pathway steps linked to direct subpathways

In [ ]:
query_next_steps = """ 
CONSTRUCT {
  ?pathwayStep bp3:nextStep ?nextStep
}
WHERE {
  VALUES ?pathwayID { "R-HSA-177929" }
  ?pathwayIDxref bp3:id ?pathwayID .
  ?pathwayIDxref rdf:type bp3:UnificationXref .
  ?pathwayStart bp3:xref ?pathwayIDxref .
  ?pathwayStart rdf:type bp3:Pathway .
  VALUES ?properties_of_interest { bp3:right bp3:left bp3:participantStoichiometry bp3:controlled bp3:controller bp3:displayName}

  ?pathwayStart bp3:pathwayComponent ?pathwayCompo .
  ?pathwayCompo bp3:pathwayOrder ?pathwayStep .
  ?pathwayStep bp3:nextStep ?nextStep .
}
"""

sparql = SPARQLWrapper(endpointURL_reactome_standalone_biopax)
sparql.setQuery(prefixes+query_next_steps)
df = save_for_cytoscape(sparql, prefixes, "../Results/10_next_steps_between_PS_linked_to_subpathways.csv", format='csv')
preview_network_data(df)

In [ ]:
q1 = pd.read_csv("../Results/1_direct_pathway_compo.csv", header=None, sep=",")
q1 = q1.drop(q1.index[0]).reset_index(drop=True)
q2 = pd.read_csv("../Results/2_direct_pathway_compo_br.csv", header=None, sep=",")
q2 = q2.drop(q2.index[0]).reset_index(drop=True)
q3 = pd.read_csv("../Results/3_direct_pathway_compo_path.csv", header=None, sep=",")
q3 = q3.drop(q3.index[0]).reset_index(drop=True)
q4 = pd.read_csv("../Results/4_direct_pathway_steps.csv", header=None, sep=",")
q4 = q4.drop(q4.index[0]).reset_index(drop=True)
q5 = pd.read_csv("../Results/5_direct_pathway_steps_next_steps_and_step_process.csv", header=None, sep=",")
q5 = q5.drop(q5.index[0]).reset_index(drop=True)
q6 = pd.read_csv("../Results/6_pathway_steps_steps_processes.csv", header=None, sep=",")
q6 = q6.drop(q6.index[0]).reset_index(drop=True)
q7 = pd.read_csv("../Results/7_linked_to_br_of_direct_pathway.csv", header=None, sep=",")
q7 = q7.drop(q7.index[0]).reset_index(drop=True)
q8 = pd.read_csv("../Results/8_processes_of_pathway_steps_linked_to_subpathways.csv", header=None, sep=",")
q8 = q8.drop(q8.index[0]).reset_index(drop=True)
q9 = pd.read_csv("../Results/9_description_processes_pf_ps_linked_to_subpathways.csv", header=None, sep=",")
q9 = q9.drop(q9.index[0]).reset_index(drop=True)
q10 = pd.read_csv("../Results/10_next_steps_between_PS_linked_to_subpathways.csv", header=None, sep=",")
q10 = q10.drop(q10.index[0]).reset_index(drop=True)


concat_df = pd.concat([q1, q2, q3, q4, q5, q6, q7, q8, q9, q10], ignore_index=True)

concat_df.to_csv("../Results/Final_pathway_egf_completed.tsv", sep="\t", header=None, index=False)
print(concat_df)